In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
action_dim = 3
sample_N = 100
Q_vs = np.random.rand(sample_N, action_dim)
Q_values = tf.constant(Q_vs, dtype=tf.float32)
Q_values.shape

In [ ]:
Q_max_values = tf.reduce_max(Q_values, axis=-1)
with tf.Session() as sess:
    print(Q_max_values.eval().shape)

In [ ]:
action_indexes = np.random.randint(action_dim, size=sample_N)
action_indexes.shape

In [ ]:
indices_d = np.dstack((np.arange(sample_N), action_indexes))
# indices

In [ ]:
# np.vstack([np.arange(sample_N), action_indexes])
indices = np.array([[i, action] for i, action in enumerate(action_indexes)])
# indices

In [ ]:
assert np.all(indices_d == indices)
assert np.all(indices_d == np.stack((np.arange(sample_N), action_indexes), axis=1))

In [ ]:
with tf.Session() as sess:
    assert np.all(tf.stack((tf.range(sample_N), action_indexes), axis=-1).eval() == indices)

In [ ]:
with tf.Session() as sess:
    print(tf.gather_nd(Q_values, indices_d).eval().shape)

In [ ]:
done_mask_ph = np.random.rand(1, sample_N)
done_mask_ph = done_mask_ph > 0.5
done_mask_ph = done_mask_ph.astype(int)[0]
done_mask_ph

In [ ]:
inputs = np.array(np.arange(action_dim * sample_N).reshape(-1, action_dim))
inputs.shape

In [ ]:
with tf.Session() as sess:
    for axis_index, shape in enumerate(inputs.shape):
        print(axis_index, shape)
        if axis_index == 1:
            print("axis=%s ==> %s" % (axis_index, tf.gather(inputs, [1], axis=axis_index).eval().shape))
        else:
            print("axis=%s ==> %s" % (axis_index, tf.gather(inputs, [0, 2], axis=axis_index).eval().shape))

In [ ]:
# using tf.one_hot for Q_values filter
with tf.Session() as sess:
    print(tf.one_hot(action_indexes, action_dim).eval().shape)
#     print(tf.one_hot(action_indexes, action_dim) * Q_values)
    print(tf.multiply(tf.one_hot(action_indexes, action_dim), Q_values).eval().shape)
#     as tf.one_hot will remove all actions with zero values
    print(tf.reduce_sum(tf.one_hot(action_indexes, action_dim) * Q_values, axis=-1).eval().shape)
    print(tf.reduce_sum(tf.one_hot(action_indexes, action_dim) * Q_values, axis=0).eval().shape)

# Q-Network Analysis

In [ ]:
import gym
import random
import os.path as osp

from gym import wrappers
from atari_wrappers import *
from dqn_utils import *

def set_global_seeds(i):
    try:
        import tensorflow as tf
    except ImportError:
        pass
    else:
        tf.set_random_seed(i)
    np.random.seed(i)
    random.seed(i)

def wrap_deepmind(env):
    assert 'NoFrameskip' in env.spec.id
    env = EpisodicLifeEnv(env)
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = ClippedRewardsWrapper(env)
    return env

In [ ]:
env = gym.make('PongNoFrameskip-v4')

seed = random.randint(0, 9999)
set_global_seeds(seed)
env.seed(seed)

expt_dir = '/tmp/hw3_vid_dir2/'
env = wrappers.Monitor(env, osp.join(expt_dir, "gym"), force=True)
env = wrap_deepmind(env)

In [ ]:
replay_buffer_size=1000000
frame_history_len=4
lander=False

replay_buffer = ReplayBuffer(replay_buffer_size, frame_history_len, lander=lander)

In [ ]:
last_obs = env.reset()
replay_buffer_idx = replay_buffer.store_frame(last_obs)
obs_Q_input = replay_buffer.encode_recent_observation()